<a href="https://colab.research.google.com/github/armandordorica/Portfolio_Optimization/blob/master/Markowitz_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L5_Data.csv  sample_data/


# **1. Classic Markowitz - Fixed Required Return Minimized Variance Portfolio**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 10 16:08:38 2019

@author: Rosario
Classic Markowitz for Fixed Required Return w Minimizing Variance Portfolio
"""

import numpy as np
import pandas as pd
import scipy.optimize as sco

# load data
data= pd.read_csv('L5_Data.csv', index_col =0)


In [ ]:
data

,AAPL,MSFT,YHOO,DB,GLD
Date,,,,,
2010-01-04,28.588685,26.593966,17.100000,61.392277,109.800003
2010-01-05,28.638111,26.602557,17.230000,62.565957,109.699997
2010-01-06,28.182584,26.439299,17.170000,61.484485,111.510002
2010-01-07,28.130486,26.164338,16.700001,62.180321,110.820000
2010-01-08,28.317506,26.344781,16.700001,64.427088,111.370003
...,...,...,...,...,...
2014-09-08,96.706827,45.257913,41.810001,34.039067,120.730003
2014-09-09,96.343043,45.540346,40.779999,33.834365,120.870003
2014-09-10,99.302455,45.618262,41.139999,34.399735,120.260002


In [ ]:

#Compute the (log) returns from prices.
#returns=np.log(data/data.shift(1)) #do not use
returns=(data-data.shift(1))/data.shift(1)

#Get the number of assets as a variable.
no_assets=len(returns.columns.tolist())

# Application: Minimize variance of a portfolio
# subject to a target return: TargetRet

# A portfolio-building function
def portfolio(weights):
    weights = np.array(weights)
    P_ret = np.sum(returns.mean()*weights)*252
    P_vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*252, weights)))
    return np.array([P_ret,P_vol, P_ret/P_vol])



#The required return is in `TargetRet`


In [ ]:
TargetRet=[0.08]


# Constraints 
**Set up the constraint that the portfolio return is equal to the TargetRet and the portfolio weights add up to one.**

In [ ]:
cons=({'type':'eq', 'fun':lambda x: portfolio(x)[0]-TargetRet},
      {'type':'eq', 'fun':lambda x: np.sum(x)-1})


**Set up boundaries for the portfolio weights (between 0 and 1) (=no short selling)**

In [ ]:
bnds=tuple((0,1) for x in range(no_assets))

#Optimisation function.
#scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
#fun : callable
#The objective function to be minimized.
#in this case: potfolio(x)[1] = P_vol 
#x0 : ndarray, shape (n,)
#Initial guess. Array of real elements of size (n,), where ‘n’ is the number of independent variables.
#In this case: [0.2, 0.2, 0.2, 0.2, 0.2]
#method : str or callable, optional
#Type of solver
#In this case: Sequential Least SQuares Programming (SLSQP)
#bounds : sequence or Bounds, Sequence of (min, max) pairs for each element in x. 
#in this case: bnds
#constraints : {Constraint, dict} 
#Equality constraint means that the constraint function result is to be zero whereas 
#inequality means that it is to be non-negative


#Optimisation function - we minimize the variance 

In [ ]:
result=sco.minimize(lambda x: portfolio(x)[1], no_assets*[1.0/no_assets], 
                 method='SLSQP', bounds=bnds, constraints=cons)


In [ ]:
answer = portfolio(result['x'])

In [ ]:
   ##The weights of the minimum volatilty portfolio having the required return is:
FinalWeights = result['x']
print("Final Weights=", FinalWeights) #final weights
print("Portfolio's return : {} \n Portfolio's volatility :{}, \n Portfolio's Sharpe Ratio:{} ".format(answer[0], answer[1], answer[2])) # statistics  P_ret,P_vol, P_ret/P_vol



Final Weights= [0.00899536 0.27650849 0.07623892 0.0041522  0.63410502]
Portfolio's return : 0.07999999999356923 
 Portfolio's volatility :0.13886528013359659, 
 Portfolio's Sharpe Ratio:0.5760979268295466 


# **2.Classic Markowitz - Fixed Required Return Minimized Variance Portfolio - Efficient Frontier**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sco

# load data
data= pd.read_csv('L5_Data.csv', index_col =0)
data

,AAPL,MSFT,YHOO,DB,GLD
Date,,,,,
2010-01-04,28.588685,26.593966,17.100000,61.392277,109.800003
2010-01-05,28.638111,26.602557,17.230000,62.565957,109.699997
2010-01-06,28.182584,26.439299,17.170000,61.484485,111.510002
2010-01-07,28.130486,26.164338,16.700001,62.180321,110.820000
2010-01-08,28.317506,26.344781,16.700001,64.427088,111.370003
...,...,...,...,...,...
2014-09-08,96.706827,45.257913,41.810001,34.039067,120.730003
2014-09-09,96.343043,45.540346,40.779999,33.834365,120.870003
2014-09-10,99.302455,45.618262,41.139999,34.399735,120.260002


**Compute the (log) returns from prices**


In [ ]:

returns=(data-data.shift(1))/data.shift(1)


**Get the number of assets as a variable**


In [ ]:

no_assets=len(returns.columns.tolist())

# Problem: Minimize variance of a portfolio
# subject to a target return: TargetRet

# A portfolio-building function
def portfolio(weights):
    weights = np.array(weights)
    P_ret = np.sum(returns.mean()*weights)*252
    P_vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*252, weights)))
    return np.array([P_ret,P_vol, P_ret/P_vol])


**The required return is: `TargetRet=[0.08]`**

In [ ]:
TargetRet=[0.08]


**Set up the constraint that the portfolio return is equal to the TargetRet and the portfolio weights add up to one**

In [ ]:
cons=({'type':'eq', 'fun':lambda x: portfolio(x)[0]-TargetRet},
      {'type':'eq', 'fun':lambda x: np.sum(x)-1})

#Set up boundaries for the portfolio weights (between 0 and 1) (=no short selling)
bnds=tuple((0,1) for x in range(no_assets))


In [ ]:

#Optimisation function.
#scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
#fun : callable
#The objective function to be minimized.
#in this case: potfolio(x)[1] = P_vol 
#x0 : ndarray, shape (n,)
#Initial guess. Array of real elements of size (n,), where ‘n’ is the number of independent variables.
#In this case: [0.2, 0.2, 0.2, 0.2, 0.2]
#method : str or callable, optional
#Type of solver
#In this case: Sequential Least SQuares Programming (SLSQP)
#bounds : sequence or Bounds, Sequence of (min, max) pairs for each element in x. 
#in this case: bnds
#constraints : {Constraint, dict} 
#Equality constraint means that the constraint function result is to be zero whereas 
#inequality means that it is to be non-negative

#Optimisation function.
#we minimize the variance 
result=sco.minimize(lambda x: portfolio(x)[1], no_assets*[1.0/no_assets], 
                 method='SLSQP', bounds=bnds, constraints=cons)


##The weights of the minimum volatilty portfolio having the required return is:
FinalWeights = result['x']
print("Final Weights=", FinalWeights) #final weights
print("P_ret,P_vol,P_ret/P_vol=", portfolio(result['x'])) # statistics  P_ret,P_vol, P_ret/P_vol

Final Weights= [5.63564030e-01 1.00753426e-01 2.56675574e-01 1.81620805e-17
 7.90069707e-02]
P_ret,P_vol,P_ret/P_vol= [0.25       0.20175483 1.23912769]


# Efficient frontier plot
* Problem: Minimize variance of a portfolio subject to a target return:TargetRet
* We solve this problem for many levels of the target return (in a loop)
* Each time we do, we get another point in the efficient frontier
* We need to specify the constraint in a loop, as the target return is always changing.


In [ ]:

#Define a range for target returns.
TargetRet=np.linspace(0.0,0.25,50)

#Define an (empty) vector for the corresponding minimum volatilities.
MinVols=[]

#Efficient Frontier Optimization Loop

#In a loop of target returns, minimise standard deviation (volatility) under the
#constraint that the portfolio return equals the target return.

for tret in TargetRet:
    #constraints:
    #portfolio return (see P_ret above) should equal target return, sum of weights x equals 1
    cons=({'type':'eq', 'fun':lambda x: portfolio(x)[0]-tret},
          {'type':'eq', 'fun':lambda x: np.sum(x)-1})
    #optimization function
    #the objective function to be minimized is the portfolio function in particular,
    #the vlatility output: P_vol above)
    res=sco.minimize(lambda x: portfolio(x)[1], no_assets*[1.0/no_assets], 
                     method='SLSQP', bounds=bnds, constraints=cons)
    MinVols.append(res['fun'])
    print(res)

MinVols=np.array(MinVols)

#Efficient Frontier Optimization Results

plt.clf()

#Efficient Frontier Plot
plt.figure(figsize=(9,6))
plt.scatter(MinVols, TargetRet, marker='x')
plt.grid()
plt.xlabel('Volatility', fontsize=18)
plt.ylabel('Expected return', fontsize=18)
plt.savefig('F2.pdf', bbox='tight')



**Showing the optimal weights of each of the five securities**

In [ ]:
res['x']

array([5.63564030e-01, 1.00753426e-01, 2.56675574e-01, 1.81620805e-17,
       7.90069707e-02])

# 3. Classic Markowitz - Minimum Variance Portfolio

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 10 16:08:38 2019

Classic Markowitz for Minimum Variance Portfolio (no fixed return required)

@author: Rosario
"""

import numpy as np
import pandas as pd
import scipy.optimize as sco

# load data
data= pd.read_csv('L5_Data.csv', index_col =0)

#Compute the (log) returns from prices.
#returns=np.log(data/data.shift(1)) #do not use
returns=(data-data.shift(1))/data.shift(1)

#Get the number of assets as a variable.
no_assets=len(returns.columns.tolist())

#Application: Minimum variance portfolio
#Minimum variance portfolio 

# A portfolio-building function
def portfolio(weights):
    weights = np.array(weights)
    P_ret = np.sum(returns.mean()*weights)*252
    P_vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*252, weights)))
    return np.array([P_ret,P_vol, P_ret/P_vol])

#Define a function for the portfolio variance.
#Variance is the square of the volatility a.k.a. standard deviation
def Variance(weights):
	return portfolio(weights)[1]**2

#Set up the constraint that portfolio weights add up to one.
cons=({'type':'eq', 'fun':lambda x: np.sum(x)-1})
#Set up boundaries for the portfolio weights (between 0 and 1) (=no short selling)
bnds=tuple((0,1) for x in range(no_assets))

#Optimisation function.
#scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
#fun : callable
#The objective function to be minimized.
#in this case: Variance 
#x0 : ndarray, shape (n,)
#Initial guess. Array of real elements of size (n,), where ‘n’ is the number of independent variables.
#In this case: [0.2, 0.2, 0.2, 0.2, 0.2]
#method : str or callable, optional
#Type of solver
#In this case: Sequential Least SQuares Programming (SLSQP)
#bounds : sequence or Bounds, Sequence of (min, max) pairs for each element in x. 
#in this case: bnds
#constraints : {Constraint, dict} 
#Equality constraint means that the constraint function result is to be zero whereas 
#inequality means that it is to be non-negative

#Optimisation function.
#we minimize the variance
result = sco.minimize(Variance, no_assets*[1.0/no_assets], method='SLSQP', bounds=bnds, constraints=cons)

#Print optimized minimum Variance portfolio weights.
print("weights= ", result['x'].round(3)) # weights
#Minimum variance portfolio properties.
print("P_ret,P_vol,P_ret/P_vol=", portfolio(result['x'])) # statistics  P_ret,P_vol, P_ret/P_vol


weights=  [0.107 0.248 0.109 0.    0.536]
P_ret,P_vol,P_ret/P_vol= [0.11071119 0.1348326  0.82110102]


# 4. Classic Markowitz - Maximum Sharpe Portfolio or Market Portfolio 

In [ ]:

import numpy as np
import pandas as pd
import scipy.optimize as sco

# load data
data= pd.read_csv('L5_Data.csv', index_col =0)

#Compute the (log) returns from prices.
#returns=np.log(data/data.shift(1)) #do not use
returns=(data-data.shift(1))/data.shift(1)

#Get the number of assets as a variable.
no_assets=len(returns.columns.tolist())

#Application: Minimum variance portfolio
#Minimum variance portfolio 


#Let the risk-free rate, r_f = 1%.
r_f = .01

# A portfolio-building function
def portfolio_CAPM(weights, r_f):
    weights = np.array(weights)
    P_ret = np.sum(returns.mean()*weights)*252
    P_vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*252, weights)))
    return np.array([P_ret,P_vol, (P_ret-r_f)/P_vol]) #modified to include risk-free rate

def Sharpe_CAPM(weights):
    return -portfolio_CAPM(weights, r_f)[2]

#Set up the constraint that portfolio weights add up to one.
cons=({'type':'eq', 'fun':lambda x: np.sum(x)-1})
#Set up boundaries for the portfolio weights (between 0 and 1) (=no short selling)
bnds=tuple((0,1) for x in range(no_assets))

#Optimisation function.
#scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
#fun : callable
#The objective function to be minimized.
#in this case: Sharpe
#x0 : ndarray, shape (n,)
#Initial guess. Array of real elements of size (n,), where ‘n’ is the number of independent variables.
#In this case: [0.2, 0.2, 0.2, 0.2, 0.2]
#method : str or callable, optional
#Type of solver
#In this case: Sequential Least SQuares Programming (SLSQP)
#bounds : sequence or Bounds, Sequence of (min, max) pairs for each element in x. 
#in this case: bnds
#constraints : {Constraint, dict} 
#Equality constraint means that the constraint function result is to be zero whereas 
#inequality means that it is to be non-negative

#Optimisation function.
#we minimize the variance
result = sco.minimize(Sharpe_CAPM, no_assets*[1.0/no_assets], method='SLSQP', bounds=bnds, constraints=cons)

#Print optimized minimum Variance portfolio weights.
print("weights= ",result['x'].round(3)) # weights
#Minimum variance portfolio properties.
print("P_ret,P_vol,P_ret/P_vol= ", portfolio_CAPM(result['x'], r_f)) # statistics  P_ret,P_vol, P_ret/P_vol


weights=  [0.615 0.083 0.274 0.    0.028]
P_ret,P_vol,P_ret/P_vol=  [0.26574934 0.21469091 1.19124438]
